In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow

In [4]:
# set the dagshub tracking server

mlflow.set_tracking_uri("https://dagshub.com/iamaniket0/UberPrediction.mlflow")

In [5]:
import dagshub
dagshub.init(repo_owner='iamaniket0', repo_name='UberPrediction', mlflow=True)

Accessing as iamaniket0

Initialized MLflow to track repo "iamaniket0/UberPrediction"

Repository iamaniket0/UberPrediction initialized!

In [6]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [7]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [8]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [9]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [10]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [11]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [12]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [13]:
from sklearn import set_config

set_config(transform_output="pandas")

In [14]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [15]:
encoder

,transformers,"[('ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [16]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

/Users/aniket_ml/UberPrediction/myenv/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [17]:
import optuna
import tqdm 

/Users/aniket_ml/UberPrediction/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# set the experiment

mlflow.set_experiment("Model Selection")

2025/09/14 18:48:04 INFO mlflow.tracking.fluent: Experiment with name 'Model Selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/915c9a343ac14af1b6f3db91348d6159', creation_time=1757868484596, experiment_id='0', last_update_time=1757868484596, lifecycle_stage='active', name='Model Selection', tags={}>

In [19]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())
        
        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [20]:
# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)

[I 2025-09-14 18:48:05,571] A new study created in memory with name: model_selection


🏃 View run rambunctious-lamb-66 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/081fd201c7204c08a14715ddd467804d
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run sedate-snipe-79 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/e39c175460864338a47bf8ffaa828f99
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run incongruous-steed-22 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/2f44c6a9b42a4549b0ac5a26db98455f
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0


[I 2025-09-14 18:49:01,291] Trial 2 finished with value: 1.5199762655746119 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 40, 'learning_rate_gb': 0.04034158883435657}. Best is trial 2 with value: 1.5199762655746119.


🏃 View run marvelous-robin-969 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/4ce241bda61f4a0a82eeadda5649b12e
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run upbeat-kit-696 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/d4ec7a939acd47978f72e11cd0362c19
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run gifted-turtle-830 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/15864816e24f44158b27cc1c8fa75f1f
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run bemused-fish-499 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/46091588b1674101a9e5b7a83e157aaa
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run righteous-sow-11 at: https://dagshub.com/iamaniket0/UberPrediction.

[I 2025-09-14 18:49:08,272] Trial 6 finished with value: 0.07934790285463009 and parameters: {'model_name': 'LR'}. Best is trial 6 with value: 0.07934790285463009.


...

🏃 View run bright-cub-119 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/c9d75ffc0ef94367b6b0d2e6d271254f
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run selective-toad-179 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/0dca2c3762b348fbb6c6346e8cf58230
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run crawling-mink-846 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/085f52fa328645a48d5bea6cbf39477a
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0
🏃 View run nosy-kite-855 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/fd2398781869440c8a11c16edf273250
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0


[I 2025-09-14 18:54:31,293] Trial 44 finished with value: 6.514469570604408 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.00010215012731339526}. Best is trial 6 with value: 0.07934790285463009.


🏃 View run honorable-roo-606 at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/db178b2de7d049bb8dfbc300c57d36ef
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0


[I 2025-09-14 18:54:33,260] Trial 45 finished with value: 6.495088314502716 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.00013374944518244254}. Best is trial 6 with value: 0.07934790285463009.
[I 2025-09-14 18:54:34,262] Trial 46 finished with value: 6.507630342760293 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.0001132929282252049}. Best is trial 6 with value: 0.07934790285463009.
[I 2025-09-14 18:54:35,245] Trial 47 finished with value: 6.510000960779919 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.00010942925586866839}. Best is trial 6 with value: 0.07934790285463009.
[I 2025-09-14 18:54:36,257] Trial 48 finished with value: 6.490714935657335 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 100, 'learning_rate_gb': 0.000140880181135276}. Best is trial 6 with value: 0.07934790285463009.
[I 2025-09-14 18:54:37,262] Trial 49 finished with value: 6.496377011577396

🏃 View run best_model at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0/runs/4fbcb8590a9743898178898d79b8e2fa
🧪 View experiment at: https://dagshub.com/iamaniket0/UberPrediction.mlflow/#/experiments/0


In [21]:
# best value

study.best_value

0.07934790285463009

In [22]:
# best parameters

study.best_params

{'model_name': 'LR'}

In [23]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

params_model_name
LR      24
GBR     13
RF      10
XGBR     3
Name: count, dtype: int64

In [24]:

from optuna.visualization import (
    plot_optimization_history, 
    plot_parallel_coordinate, 
    plot_param_importances
)

In [28]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [ ]:
plot_parallel_coordinate(study, params=["model_name"])

In [ ]:
# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded) 
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

In [ ]:
lr.coef_

In [ ]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss
        

In [ ]:
# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

In [ ]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

In [ ]:
# best parameters

study.best_params

In [ ]:
# best value

study.best_value